<a href="https://colab.research.google.com/github/C241-PS517-ISALAT/Isalat-Model/blob/model_test_1/model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [23]:
from google.colab import drive

import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D , MaxPooling2D , Flatten , Dropout , BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import numpy as np
import matplotlib.pyplot as plt
import scipy as sp
import cv2

In [8]:
drive.mount('/content/drive')
train_data = "/content/drive/My Drive/bisindo/data/train"
validation_data = "/content/drive/My Drive/bisindo/data/validation"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [9]:
# Define image dimensions (adjust if needed)
img_width, img_height = 256, 256  # Example image dimensions

# Create ImageDataGenerator for training data
train_datagen = ImageDataGenerator(
    rescale=1./255,  # Normalize pixel values
    shear_range=0.2,  # Randomly apply shearing transformation
    zoom_range=0.2,  # Randomly apply zooming
    horizontal_flip=True  # Randomly flip images horizontally
)

# Create ImageDataGenerator for validation data (without data augmentation)
val_datagen = ImageDataGenerator(rescale=1./255)

# Generate augmented images
train_generator = train_datagen.flow_from_directory(
    train_data,
    target_size=(img_width, img_height),
    batch_size=32,
    class_mode='sparse',
    save_to_dir='/content/drive/My Drive/bisindo/data/augmented_images',
    save_prefix='aug',
    save_format='jpeg'
)

validation_generator = val_datagen.flow_from_directory(
    validation_data,
    target_size=(img_width, img_height),
    batch_size=32,
    class_mode='sparse'
)

Found 234 images belonging to 26 classes.
Found 78 images belonging to 26 classes.


In [10]:
# Generate at least 500 images
num_augmented_images = 0
while num_augmented_images < 500:
    images, labels = next(train_generator)
    num_augmented_images += len(images)

In [24]:
# Define SqueezeNet architecture
##STILL NEED SOME FIXING##
def fire_module(x, squeeze_filters, expand_filters):
    x = layers.Conv2D(squeeze_filters, (1, 1), padding='valid', activation='relu')(x)
    left = layers.Conv2D(expand_filters, (1, 1), padding='valid', activation='relu')(x)
    right = layers.Conv2D(expand_filters, (3, 3), padding='same', activation='relu')(x)
    x = layers.concatenate([left, right])
    return x

input_shape = (img_width, img_height, 3)
input_img = layers.Input(shape=input_shape)

x = layers.Conv2D(96, (7, 7), strides=(2, 2), padding='valid', activation='relu')(input_img)
x = layers.MaxPooling2D(pool_size=(3, 3), strides=(2, 2))(x)
x = fire_module(x, squeeze_filters=16, expand_filters=64)
x = fire_module(x, squeeze_filters=16, expand_filters=64)
x = fire_module(x, squeeze_filters=32, expand_filters=128)
x = layers.MaxPooling2D(pool_size=(3, 3), strides=(2, 2))(x)
x = fire_module(x, squeeze_filters=32, expand_filters=128)
x = fire_module(x, squeeze_filters=48, expand_filters=192)
x = fire_module(x, squeeze_filters=48, expand_filters=192)
x = fire_module(x, squeeze_filters=64, expand_filters=256)
x = layers.MaxPooling2D(pool_size=(3, 3), strides=(2, 2))(x)
x = fire_module(x, squeeze_filters=64, expand_filters=256)
x = layers.Dropout(0.5)(x)
x = layers.Conv2D(26, (1, 1), padding='valid', activation='relu')(x)
x = layers.GlobalAveragePooling2D()(x)
output = layers.Activation('softmax')(x)

model = models.Model(input_img, output)

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


In [25]:
#model train
model.fit(train_generator, epochs=10, validation_data=validation_generator)

Epoch 1/10
8/8 [==============================] - 100s 12s/step - loss: 3.2594 - accuracy: 0.0128 - val_loss: 3.2581 - val_accuracy: 0.0385
Epoch 2/10
8/8 [==============================] - 66s 8s/step - loss: 3.2583 - accuracy: 0.0385 - val_loss: 3.2581 - val_accuracy: 0.0385
Epoch 3/10
8/8 [==============================] - 65s 8s/step - loss: 3.2582 - accuracy: 0.0385 - val_loss: 3.2581 - val_accuracy: 0.0385
Epoch 4/10
8/8 [==============================] - 65s 8s/step - loss: 3.2583 - accuracy: 0.0385 - val_loss: 3.2581 - val_accuracy: 0.0385
Epoch 5/10
8/8 [==============================] - 62s 7s/step - loss: 3.2581 - accuracy: 0.0385 - val_loss: 3.2581 - val_accuracy: 0.0385
Epoch 6/10
8/8 [==============================] - 64s 7s/step - loss: 3.2581 - accuracy: 0.0385 - val_loss: 3.2581 - val_accuracy: 0.0385
Epoch 7/10
8/8 [==============================] - 62s 7s/step - loss: 3.2581 - accuracy: 0.0385 - val_loss: 3.2581 - val_accuracy: 0.0385
Epoch 8/10
8/8 [================